# Optimizer Agent Notebook

This notebook demonstrates the Optimizer Agent's capabilities in improving quantum circuits.

## Purpose
The Optimizer Agent refines generated circuits to improve performance and efficiency. This notebook covers:

1.  **Heuristic Optimization**: Applying standard Cirq optimizations like gate merging and dropping negligible operations.
2.  **RL Optimization**: Using a Reinforcement Learning loop to iteratively improve circuit metrics (depth, gate count) based on a reward function.
3.  **Comparison**: Analyzing and comparing the metrics of the original vs. optimized circuits.

## Usage
Run this notebook to optimize existing Cirq circuits and observe the improvements in circuit depth and gate count.


In [ ]:
import sys
import os
from pathlib import Path
import cirq

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.cirq_rag_code_assistant.config import get_config, setup_logging
from src.agents.optimizer import OptimizerAgent

# Setup logging
setup_logging()

### Initialize Agent
Initialize the Optimizer Agent.

In [ ]:
optimizer = OptimizerAgent()
print("Optimizer Agent initialized.")

### Create Sample Circuit
Let's create an unoptimized circuit to test.

In [ ]:
# Create a circuit with redundant gates
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.X(q0),
    cirq.X(q0),  # Redundant (Identity)
    cirq.Z(q1),
    cirq.Z(q1),  # Redundant (Identity)
    cirq.CNOT(q0, q1),
    cirq.CNOT(q0, q1), # Redundant (Identity)
    cirq.H(q0),
    cirq.H(q0)   # Redundant (Identity)
)

print("Original Circuit:")
print(circuit)

### Optimize Circuit
Now let's run the optimizer.

In [ ]:
task = {
    "circuit": circuit,
    "optimization_level": "aggressive",
    "use_rl": True  # Enable RL optimization
}

try:
    result = optimizer.execute(task)
    
    if result['success']:
        print("Successfully optimized circuit!")
        print("\nOptimized Circuit:")
        print("-" * 40)
        print(result['optimized_code'])
        print("-" * 40)
        
        print("\nMetrics Comparison:")
        print(f"Original Depth: {result['original_metrics'].get('depth')}")
        print(f"Optimized Depth: {result['optimized_metrics'].get('depth')}")
        print(f"Original Gate Count: {result['original_metrics'].get('total_gate_count')}")
        print(f"Optimized Gate Count: {result['optimized_metrics'].get('total_gate_count')}")
        
        print("\nImprovements:")
        for imp in result['improvements']:
            print(f"- {imp}")
            
    else:
        print(f"Failed to optimize circuit: {result.get('error')}")
        
except Exception as e:
    print(f"Error executing task: {e}")